
## **Zookeeper**
Kafka는 2.8.0 이전 버전까지는 공통적으로 Zookeeper를 코디네이트 도구로 사용한다. Zookeeper가 Kafka에서 수행하는 역할은 크게 5가지이다.

1. **Controller Election** : controller는 각 파티션에 대해 leader/follower 관계를 관리하는 broker이다. Zookeeper는 만약 어느 broker 노드가 shut down될 경우, shut down된 노드의 leader 파티션을 다른 노드의 replica가 대체할 것을 보장해준다.

2. **Cluster Membership** : Zookeeper는 클러스터 상의 정상적인 broker 리스트를 관리한다.

3. **Topic Configuration** : Zookeeper는 모든 토픽의 configuration을 관리한다. 현재 유효한 토픽들 리스트, 각 토픽의 파티션 수, replica들의 위치, 토픽별 configuration override 등의 정보를 관리한다.

4. **Access Control Lists (ACLs)** : Zookeeper는 모든 토픽에 대해 접근제어목록(ACL)을 관리한다. 토픽 별 read/write 허용 사용자 리스트, consumer group 리스트, 그룹별 멤버 리스트, consumer group별 각 파티션에 대한 최신 offset 정보 등을 관리한다.
5. **Quotas (허용량)** : Zookeeper는 각 클라이언트가 데이터를 얼마나 read/write 할 수 있는지에 대한 허용량을 관리한다.




## Zookeeper 최적화

- 대부분의 경우, 8GB RAM이면 Zookeeper를 구동하기에 충분하다.
- Zookeeper를 위한 CPU 코어를 할당함으로써 Context Switching 이슈가 발생하지 않도록 한다.
- SSD를 사용하여 Disk write latency를 낮춘다.





## **Kafka's Partition 간단요약**
Kafka에 메시지를 병렬 처리 방식으로 보내고 받을 수 있도록 하기 위해 Topic당 여러개의 partition을 할당할 수 있다.
메시지를 produce/consume하는 프로세스 수 만큼 partition을 생성한다면 보다 효율적으로 메시지를 주고받을 수 있다.
Kafka에 메시지를 read/write하는 작업도 엄연히 시간을 소요하는 I/O 작업이다.

## Too many Partitions
Topic의 partition 개수를 늘리면 빠른 전송이 가능하다.
그러나 무조건 partition 수가 많은 것이 좋은 것은 아니다.
과도하게 많은 partition 수는 Kafka의 성능에 악영향을 준다.

- <b>파일 핸들러 낭비</b>
  - 각 partition은 broker의 디렉토리와 매핑된다. 즉, <b>partition 수가 많을 수록 Kafka는 많은 파일 핸들</b>을 열게 되고, 이는 리소스 낭비를 유발한다.
- <b>장애 복구 시간 증가</b>
  - Kafka는 장애 복구를 위해 replication을 지원하며, 각 partition 별로 replica를 생성한다.
  - partition의 replica들은 leader와 follower로 구분되며 오직 leader만 읽기/쓰기를 담당한다.
  - 노드 장애 시 <b>broker controller</b>는 각 파티션별 leader를 새로 선출한다. partition 수가 많을 수록 선출 작업은 오래 걸린다.
  - <b>broker controller</b>가 다운될 경우, 
  남은 broker가 새로운 controller가 되고 초기화되는 동안 zookeeper는 <b>모든 partition의 데이터를 읽어야 한다</b>.